In [1]:
# Notebook to evaluate active optics strategies

In [1]:
import numpy as np
import logging
import pickle
from scipy.linalg import block_diag
import scipy.io as spio
# required libraries for Reconstructor matrix computation or MPC
from scipy import sparse

import os.path
from os import path

from pathlib import Path
# import DOS library
import dos

from ruamel import yaml
from ruamel.yaml import YAML
y = YAML()
y.default_flow_style = None

sim_folder = 'dos/SH_AcO'
calibDataFolder = '/Users/rromano/Workspace/SIMCEO/calibration_dbs'

In [2]:
# Load data from yaml file

with open(Path(sim_folder)/'wfs48.yaml','r') as stream:
        wfs48_dt = yaml.safe_load(stream)
# with open(Path(sim_folder)/'AcO_controller.yaml','r') as stream:
#         aco_ctrl_dt = yaml.safe_load(stream)

# Calibrations data
#dt = np.load(Path(calibDataFolder)/'calib_dt_SH0p5fluxTH_46BM.npz', allow_pickle=True)
dt = np.load(Path(calibDataFolder)/'calib_dt_GSH0p5fluxTH_46BM.npz', allow_pickle=True)

In [3]:
def get_aco_recW(Dsh,n_bm,rho_F):
    
    # --- W2 ---
    _U,sigma,VT = np.linalg.svd(Dsh,full_matrices=False)
    n_r = 12
    q = sigma[-n_r:] #40*sigma[-12:] #np.ones((12)) #np.zeros_like(sigma)
    W2 = VT[-n_r:,:].T.dot(np.diag(q)).dot(VT[-n_r:,:])

    # --- W3 ---
    #rho_F = 1.
    # RBMs
    W_M1TxyzRxyz = 4*np.ones(6)
    W_M2TxyzRxyz = np.ones(6)   
    W_rbm_oa = block_diag(np.diag(W_M1TxyzRxyz), np.diag(W_M2TxyzRxyz))
    W_rbm_cs = block_diag(np.diag(W_M1TxyzRxyz[:-1]), np.diag(W_M2TxyzRxyz[:-1]))
    
    # OA segment weights
    W_m1oaFz = np.diag([1]*165)
    W_m1oaFz =  rho_F*(1/np.linalg.norm(W_m1oaFz))*W_m1oaFz
    # CS (center segment) weights
    W_m1csFz = np.diag([1]*154)
    W_m1csFz =  rho_F*(1/np.linalg.norm(W_m1csFz))*W_m1csFz


    # Rescale RBM weighting matrix
    rbm_factor = 1/np.linalg.norm(W_rbm_oa) 
    # Merge weighting matrices
    W3_oa = block_diag(rbm_factor*W_rbm_oa,W_m1oaFz)
    W3_cs = block_diag(rbm_factor*W_rbm_cs,W_m1csFz)
    W3 = block_diag(np.kron(np.eye(6),W3_oa),W3_cs)
    
    W3 = (1/np.linalg.norm(W3))*W3
    
    return W2, W3


In [4]:
n_bm = 27
K = 0.25

#### AcO reconstructor+controller settings

In [5]:
# *** Reconstructor tuning settings ***

# Number of bending modes
n_bm = 27

# Consolidate SH-WFS interaction matrices of all segments
Dsh = block_diag(*[Dseg[:,:12+n_bm] for Dseg in dt['calibdt'][()]['D'][:-1]])
Dsh = block_diag(Dsh,dt['calibdt'][()]['D'][-1][:,:10+n_bm])

# Max actuator command values
max_m1RBM = [3.0e-3,4.75e-3,4e-3] + [6.5e-4,5.75e-4,5.75e-4]
max_m2RBM = [3.0e-5,3.0e-5,3.0e-5] + [3.5e-3,3.5e-3,3.5e-3]
W2,W3 = get_aco_recW(Dsh,n_bm,.1*(12/165)**2)

max_Fz = 147
fz_factor = np.mean(max_m1RBM + max_m2RBM)/max_Fz
rho_2, rho_3 = 1., 1.

# Print reconstructor regularization weights
np.set_printoptions(formatter={'float': '{: 0.6f}'.format})
#aux = np.reshape(np.insert(np.diag(W3),[72+(n_bm*6)+5,72+(n_bm*6)+10],0),[7,12+n_bm])
aux = np.insert(np.diag(W3),[72+(165*6)+5,72+(165*6)+10],0)
aux = np.reshape(np.append(aux,np.array([0]*11)),[7,12+165])
_wM1rbm,_wM2rbm,_wBM = np.split(aux,[6,12],axis=1)
print('M1/2 RBM weights (M1/TxyzRxyz):\n',_wM1rbm[0,:])
print('\nM1/2 RBM weights (M2/TxyzRxyz):\n',_wM2rbm[0,:])
print('\nForce weights:\n',_wBM[0,0])

np.set_printoptions(formatter=None)

# Bending modes' data
Afz = {}

# OA segment weights
filepath = os.path.join('../ActiveOptics','Af_oa_axial_ver3.mat')
Afz['outer'] = spio.loadmat(filepath)['afprime']
_U,s_oa,V_oaT = np.linalg.svd(Afz['outer'],0)
bm2Fz_oa = np.dot(V_oaT.T, np.diag(1/s_oa)[:,:n_bm])
# CS (center segment) weights
filepath = os.path.join('../ActiveOptics','Af_cs154_axial.mat')
Afz['center'] = spio.loadmat(filepath)['afprime']
_U,s_cs,V_csT = np.linalg.svd(Afz['center'],0)
bm2Fz_cs = np.dot(V_csT.T, np.diag(1/s_cs)[:,:n_bm])

rbm_ranges = max_m1RBM+max_m2RBM
oa_ranges = np.array(rbm_ranges + [fz_factor*max_Fz]*bm2Fz_oa.shape[0])
cs_ranges = np.array(rbm_ranges + [fz_factor*max_Fz]*bm2Fz_cs.shape[0])

umax = np.hstack([np.kron(np.array([1]*6),oa_ranges),cs_ranges]) # np.array([]) #
umin = -umax
_Tu = block_diag(np.kron(np.eye(6),block_diag(np.eye(12),fz_factor*bm2Fz_oa)),
                 block_diag(np.eye(12),fz_factor*bm2Fz_cs))
_TuwoS7Rz = block_diag(np.kron(np.eye(6),block_diag(np.eye(12),fz_factor*bm2Fz_oa)),
                 block_diag(np.eye(10),fz_factor*bm2Fz_cs))


# *** Controller settings ****
K = 0.25   # Controller gain


# AcO dictionary
data = {'SHAcO_qp':{'D':Dsh,'n_bm':n_bm,
                    'W2':W2,'W3':_TuwoS7Rz.T.dot(W3).dot(_TuwoS7Rz),
                    'K':K,'rho_2':rho_2,'rho_3':rho_3,
                    'wfsMask':dt['calibdt'][()]['wfsMask'],
                    'umin':umin, 'umax':umax,'_Tu':_Tu,'J1_J3_ratio':100}}
# Pickles MPC data into string representation
with open(Path(sim_folder)/'SHAcO_qp.pickle','wb') as f:
    pickle.dump(data,f)
    

M1/2 RBM weights (M1/TxyzRxyz):
 [ 0.151511  0.151511  0.151511  0.151511  0.151511  0.151511]

M1/2 RBM weights (M2/TxyzRxyz):
 [ 0.037878  0.037878  0.037878  0.037878  0.037878  0.037878]

Force weights:
 1.5752020421330646e-05


#### Set initial M1/M2 states and generate respective yaml files

In [6]:
# Python code to set simulation states
from ruamel import yaml
from ruamel.yaml import YAML
m1m2load_folder = 'testDT/M1M2_loads'
r_id = 1

test_case = 'random' # 'random_RBM' # 'BM_dist' #

if(test_case == 'BM_dist'):
    fileSTR = '_2_3bm'
elif(test_case == 'random_RBM'):
    fileSTR = '_rand_rbm_%d' %r_id
elif(test_case == 'random'):
    fileSTR = '_rbmplusbm_%d' %r_id
else:
    fileSTR = '_noload' %r_id

print(fileSTR)

with open(Path(m1m2load_folder)/('M1'+fileSTR+'.yaml'),'r') as f:
    m1_x0_dt = yaml.safe_load(f)
with open(Path(m1m2load_folder)/('M2'+fileSTR+'.yaml'),'r') as f:
    m2_x0_dt = yaml.safe_load(f)
   
 
# Update yaml files with M1/M2 initial conditions    
with open(Path(sim_folder)/'M1.yaml','w') as f:
        y.dump(m1_x0_dt, f)
with open(Path(sim_folder)/'M2.yaml','w') as f:
        y.dump(m2_x0_dt, f)
    

In [7]:

# Set of guide star magnitudes to be investigated
mag_values = np.array([6, 8, 10, 12, 14, 16, 18, 20])

for imag in range(len(mag_values)): 
    # Set guide star magnitude
    wfs48_dt['source']['magnitude'] = (mag_values[imag]*np.array([1,1,1])).tolist()
    print('Magnitude:',wfs48_dt['source']['magnitude'])
    with open(Path(sim_folder)/'wfs48.yaml','w') as f:
        y.dump(wfs48_dt, f)

    # invoke DOS
    sim = dos.DOS(sim_folder,verbose=logging.WARNING)
    # Start simulation
    sim._run_()

    #sim_log_dt = {'sim_log_dt':sim.logs.entries, 'AcO_dt':data}
    sim_log_dt = {'sim_log_dt':sim.logs.entries}
    # Pickles interaction matrix data into string representation
    #filename = 'dt_mag%d_ctrlK%sreg100_QP_rec_test.pickle'% (mag_values[imag],str(K).replace(".", "p"))
    filename = 'dt_mag%d_ctrlK%sTSVD_rec_test.pickle'% (mag_values[imag],str(K).replace(".", "p"))
    #filename = 'dt_mag%d_ctrlK%sreg100_QP_recBMtest.pickle'% (mag_values[imag],str(K).replace(".", "p"))
    #filename = 'dt_mag%d_ctrlK%sreg100_QP_recRBMtest.pickle'% (mag_values[imag],str(K).replace(".", "p"))
    #filename = 'dt_mag%d_ctrlK%sreg20RecM_reWM2_BMtest.pickle'% (mag_values[imag],str(K).replace(".", "p"))
    #filename = 'dt_mag%d_ctrlK%sTSVD_TSVDRecM_BMtest.pickle'% (mag_values[imag],str(K).replace(".", "p"))
    #filename = 'dt_mag%d_ctrlK%sTSVDRecM.pickle'% (mag_values[imag],str(K).replace(".", "p"))
    with open(Path('testDT')/filename,'wb') as f:
        pickle.dump(sim_log_dt,f)



INFO:wfs48 data:Linked to data from wfs48
INFO:M1 Txyz:Linked to Txyz from M1
INFO:M1 Rxyz:Linked to Rxyz from M1
INFO:M2 Txyz:Linked to Txyz from M2
INFO:M2 Rxyz:Linked to Rxyz from M2
INFO:M1 BM:Linked to modes from M1


Magnitude: [6, 6, 6]
SIMCEO server received: Acknowledging connection from SIMCEO client!
27 BMs used in the reconstructor computation.
No J1_J3_ratio informed. Using TSVD reconstructor
Elapsed time: 49.577433824539185
Magnitude: [8, 8, 8]


INFO:wfs48 data:Linked to data from wfs48
INFO:M1 Txyz:Linked to Txyz from M1
INFO:M1 Rxyz:Linked to Rxyz from M1
INFO:M2 Txyz:Linked to Txyz from M2
INFO:M2 Rxyz:Linked to Rxyz from M2
INFO:M1 BM:Linked to modes from M1


SIMCEO server received: Acknowledging connection from SIMCEO client!
27 BMs used in the reconstructor computation.
No J1_J3_ratio informed. Using TSVD reconstructor
Elapsed time: 52.5614378452301
Magnitude: [10, 10, 10]


INFO:wfs48 data:Linked to data from wfs48
INFO:M1 Txyz:Linked to Txyz from M1
INFO:M1 Rxyz:Linked to Rxyz from M1
INFO:M2 Txyz:Linked to Txyz from M2
INFO:M2 Rxyz:Linked to Rxyz from M2
INFO:M1 BM:Linked to modes from M1


...

Magnitude: [16, 16, 16]
SIMCEO server received: Acknowledging connection from SIMCEO client!
27 BMs used in the reconstructor computation.
No J1_J3_ratio informed. Using TSVD reconstructor
Elapsed time: 53.65119528770447
Magnitude: [18, 18, 18]


INFO:wfs48 data:Linked to data from wfs48
INFO:M1 Txyz:Linked to Txyz from M1
INFO:M1 Rxyz:Linked to Rxyz from M1
INFO:M2 Txyz:Linked to Txyz from M2
INFO:M2 Rxyz:Linked to Rxyz from M2
INFO:M1 BM:Linked to modes from M1


SIMCEO server received: Acknowledging connection from SIMCEO client!
27 BMs used in the reconstructor computation.
No J1_J3_ratio informed. Using TSVD reconstructor
Elapsed time: 52.25102090835571
Magnitude: [20, 20, 20]


INFO:wfs48 data:Linked to data from wfs48
INFO:M1 Txyz:Linked to Txyz from M1
INFO:M1 Rxyz:Linked to Rxyz from M1
INFO:M2 Txyz:Linked to Txyz from M2
INFO:M2 Rxyz:Linked to Rxyz from M2
INFO:M1 BM:Linked to modes from M1


SIMCEO server received: Acknowledging connection from SIMCEO client!
27 BMs used in the reconstructor computation.
No J1_J3_ratio informed. Using TSVD reconstructor
Elapsed time: 51.82087969779968


0.0005289256198347108